**Prueba desempeño**

Parámetros de la prueba:

Arquitectura: Inception V3


Datos:skin-cancer-3- CLASES


Data Augmentation: No


K-Fold Validation: No

Hyperparameter Tuning: No

Parámetros estáticos de entrenamiento:


lr = 1e-3

init = 'normal'

activ = 'relu'

activ_last = 'softmax'

optim = 'adam'

epochs = 40 

batch_size = TENER CUIDADO 


In [1]:
# Librerias Basicas 
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os
import seaborn as sns
import itertools
from glob import glob
from PIL import Image
import PIL.Image
from IPython.display import Image
# Librerias de preprocesamiento y metricas
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
# Librerias para backend, arquitectura y entrenamiento
import keras
import tensorflow as tf
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers.core import Activation
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input

**LLAMAR LOS DATOS**


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# identificar las rutas y los nombres de las carpetas
folder_carcinoma_train = '../content/drive/MyDrive/data_todo/train/basal_cell_carcinoma_and_squamous_cell_carcinoma'
folder_melanoma_train = '../content/drive/MyDrive/data_todo/train/melanoma'
folder_keratosis_train= '../content/drive/MyDrive/data_todo/train/pigmented_bening_keratosis'

folder_carcinoma_test = '../content/drive/MyDrive/data_todo/Test/basal_cell_carcinoma_and_squamous_cell_carcinoma'
folder_melanoma_test = '../content/drive/MyDrive/data_todo/Test/melanoma'
folder_keratosis_test = '../content/drive/MyDrive/data_todo/Test/pigmented_bening_keratosis'

**proceso de datos**

In [4]:
read = lambda imname: np.asarray(PIL.Image.open(imname).convert("RGB"))

# cargar las imagenes de train 
ims_carcinoma = [read(os.path.join(folder_carcinoma_train, filename)) for filename in os.listdir(folder_carcinoma_train)]
X_carcinoma = np.array(ims_carcinoma, dtype='uint8')
#print (X_carcinoma)
ims_melanoma = [read(os.path.join(folder_melanoma_train, filename)) for filename in os.listdir(folder_melanoma_train)]
X_melanoma=[]
for img in ims_melanoma:
  try:
   X_melanoma.append(np.array(ims_melanoma, dtype='uint8'))
  except:
   plt.imshow(img)
ims_keratosis = [read(os.path.join(folder_keratosis_train, filename)) for filename in os.listdir(folder_keratosis_train)]
X_keratosis = np.array(ims_keratosis, dtype='uint8')



In [ ]:
# Cargar las imagenes de test
ims_carcinoma = [read(os.path.join(folder_carcinoma_test, filename)) for filename in os.listdir(folder_carcinoma_test)]
X_carcinoma_test = np.array(ims_carcinoma, dtype='uint8')
ims_melanoma = [read(os.path.join(folder_melanoma_test, filename)) for filename in os.listdir(folder_melanoma_test)]
X_melamona_test = np.array(ims_melanoma, dtype='uint8')
ims_keratosis = [read(os.path.join(folder_keratosis_test, filename)) for filename in os.listdir(folder_keratosis_test)]
X_keratosis_test = np.array(ims_keratosis, dtype='uint8')



ValueError: ignored

In [ ]:
# Crear etiquetas
y_carcinoma = np.zeros(X_carcinoma.shape[0])
y_melanoma = np.ones(X_melanoma.shape[0])
y_keratosis = 2*np.ones(X_keratosis.shape[0])


y_carcinoma_test = np.zeros(X_carcinoma_test.shape[0])
y_melanoma_test = np.ones(X_melanoma_test.shape[0])
y_keratosis_test = 2*np.ones(X_keratosis_test.shape[0])



In [ ]:
# Crear datos combinados para train y test 
X_train = np.concatenate((X_carcinoma, X_melanoma,X_keratosis), axis = 0)
y_train = np.concatenate((y_carcinoma, y_melanoma,y_keratosis), axis = 0)



X_test = np.concatenate((X_carcinoma_test, X_melanoma_test,X_keratosis_test), axis = 0)
y_test = np.concatenate((y_carcinoma_test, y_melanoma_test,y_keratosis_test), axis = 0)





In [ ]:
# Randomizar los datos
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
y_test = y_test[s]

In [ ]:
w=40
h=30
fig=plt.figure(figsize=(12, 8))
columns = 5
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if y_train[i] == 0:
        ax.title.set_text('Carcinoma')
    elif  y_train[i] == 1:
        ax.title.set_text('Melanoma')
    else:
        ax.title.set_text('Keratosis')
    plt.imshow(X_train[i], interpolation='nearest')
plt.show()

ETIQUETAS CATEGORIAS


In [ ]:
# Etiquetas a categoricas 
y_train = to_categorical(y_train, num_classes= 3)
y_test = to_categorical(y_test, num_classes= 3)

# Normalización 
X_train = X_train/255.
X_test = X_test/255.

NameError: ignored